## Trival Vector Class

In [1]:
import math
import unittest
import operator
import numbers
import random
import sys

In [2]:
class Vector(object):
    
    _EMPTY_COORDS = "The Coordinates must be non-empty"
    _EQUAL_DIM = "The Vectors must have equal dimension"
    _DIM_ERROR = "Function only defined for 3D Vector or 2D vector with set z=0"
    _MUST_BE_NUMERIC = "Coordindates must be Numeric"
    _MUST_BE_VECTOR = "Operation requires argument of type Vector"
    _NO_DIR_FOR_ZERO_VEC = "Zero vector has no defined direction"
    _NO_ANGLE_FOR_ZERO_VEC = "Angle with respect to zero vector not defined"
    _NO_PROJECTION_FOR_ZERO_VEC = "Projection onto zero vector not defined"
    
    def __init__(self, coordinates):
        if not coordinates:
            raise ValueError(Vector._EMPTY_COORDS)
               
        __coordinates = []
        for x in coordinates:
            if isinstance(x, numbers.Number) == True and x != None:
                __coordinates.append(float(x))
            else:
                raise ValueError(Vector._MUST_BE_NUMERIC)
        
        self._coordinates = tuple(__coordinates)
        self._dimension = len(__coordinates)
        self._is_zero = (self._dimension * sum([math.fabs(x) for x in self._coordinates])) == 0 
    
    def is_zero(self):
        return self._is_zero
    
    def __str__(self):
        return 'vector: {}'.format(self._coordinates)
    
    def __eq__(self, vin, tolerance=1e-10):
        return max([math.fabs(x) for x in (self-vin)._coordinates]) < tolerance
    
    # With Map
    def __add__(self,vin):
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM)   

        return Vector([x+y for x,y in zip(self._coordinates, vin._coordinates)])

    # With List Comprehension
    def __sub__(self,vin):
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM)   

        return Vector([x-y for x,y in zip(self._coordinates, vin._coordinates)])
    
    # Multiply
    def __mul__(self,vin):        
        if isinstance(vin, numbers.Number) == True:
            v = Vector([vin] * self._dimension)
        else:
            v = vin

        if self._dimension != v._dimension:
            raise ValueError(Vector._EQUAL_DIM)   

        return Vector([x*y for x,y in zip(self._coordinates, v._coordinates)])
    
    # divide
    def __truediv__(self,vin):
        return self.__div__(vin)
    
    def __div__(self,vin):
        
        if isinstance(vin, numbers.Number) == True:
            v = Vector([vin] * self._dimension)
        else:
            v = vin

        if self._dimension != v._dimension:
            raise ValueError(Vector._EQUAL_DIM)   

        return Vector([x/y for x,y in zip(self._coordinates, v._coordinates)])

    # Return the magnitude of the vector
    def norm(self):
        return (math.sqrt(sum(list(map(lambda x:x*x,self._coordinates)))))
    
    # Return the direction (unit vector)
    def direction(self):
        res = None
        try:
            res = self * (float(1)/self.norm())
        except ZeroDivisionError as e:
            raise type(e)(Vector._NO_DIR_FOR_ZERO_VEC).with_traceback(sys.exc_info()[2])

        return res

    # Check if vectors are parallel
    def is_parallel_to(self,vin,tolerance=1e-10):
        if vin.__class__.__name__ != self.__class__.__name__:
            raise TypeError(Vector._MUST_BE_VECTOR)
        if self.is_zero() or vin.is_zero():
            return True # Zero vec is parallel to all
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM) 

        return ((self.angle(vin) < tolerance) or (math.fabs(self.angle(vin)-math.pi) < tolerance))
    
    # Return the dot product.
    def dot(self,vin):
        if vin.__class__.__name__ != self.__class__.__name__:
            raise TypeError(Vector._MUST_BE_VECTOR)
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM)       

        return sum((self * vin)._coordinates)

    def _clip(x):
        r = x
        if(r>float(1)):
            r=float(1)
        if(r<float(-1)):
            r=float(-1)
        return r

    def __neg__(self):
        return self * float(-1)

    # Return the angle w.r.t the given vector
    def angle(self,vin):
        if vin.__class__.__name__ != self.__class__.__name__:
            raise TypeError(Vector._MUST_BE_VECTOR)
        if self.is_zero() or vin.is_zero():
            raise ValueError(Vector._NO_ANGLE_FOR_ZERO_VEC)
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM) 

        vs = self.direction()
        v = vin.direction()
       
        return math.acos(Vector._clip(vs.dot(v)))

    # Is orthogonal if angle = pi/2
    def is_orthogonal_to(self,vin,tolerance=1e-10):
        if vin.__class__.__name__ != self.__class__.__name__:
            raise TypeError(Vector._MUST_BE_VECTOR)
        if self.is_zero() or vin.is_zero():
            return True # Zero vec is orthogonal to all
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM) 

        return math.fabs(self.dot(vin)) < tolerance
    
    def projection_onto(self,vin):
        if vin.__class__.__name__ != self.__class__.__name__:
            raise TypeError(Vector._MUST_BE_VECTOR)
        if self.is_zero() or vin.is_zero():
            raise ValueError(Vector._NO_PROJECTION_FOR_ZERO_VEC)
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM) 

        v = vin.direction()
       
        return (v*(self.dot(v)))
    
    def orth_projection_onto(self,vin):
        if vin.__class__.__name__ != self.__class__.__name__:
            raise TypeError(Vector._MUST_BE_VECTOR)
        if self.is_zero() or vin.is_zero():
            raise ValueError(Vector._NO_PROJECTION_FOR_ZERO_VEC)
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM) 

        return self - self.projection_onto(vin)
    
    def cross_product(self,vin):
        if vin.__class__.__name__ != self.__class__.__name__:
            raise TypeError(Vector._MUST_BE_VECTOR)
        if self.is_zero():
            return self # cross with zero is the zero vector
        if vin.is_zero():
            return vin # cross with zero is the zero vector
        if self._dimension != vin._dimension:
            raise ValueError(Vector._EQUAL_DIM) 
        if self._dimension == 2:
            [x1,y1] = self._coordinates
            [x2,y2] = vin._coordinates
            z1 = 0
            z2 = 0
        elif self._dimension == 3:
            [x1,y1,z1] = self._coordinates
            [x2,y2,z2] = vin._coordinates
        else:
            raise ValueError(Vector._DIM_ERROR)
        
        return Vector([ (y1*z2 - y2*z1),
                       -(x1*z2 - x2*z1),
                        (x1*y2 - x2*y1)])


In [46]:
class testVector(unittest.TestCase):

    def _rnd_data():
        random.seed(42)
        return list(map(lambda x: float(x)*float(1), [random.uniform(-10,10) for _ in range(4)]))
    
    def _pi(mul=float(1)):
        return math.pi*mul
    
    def test_init(self):
        print("\nTest Vector Init")
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[False],[int(0)],True],
                     [[True],[int(1)],True],
                     [[False],[float(0)],True],
                     [[True],[float(1)],True],
                     [[False]*5,[int(0)]*5,True],
                     [[True]*6,[int(1)]*6,True],
                     [[False]*7,[float(0)]*7,True],
                     [[True]*8,[float(1)]*8,True],
                     [[True,False,False,True,False,True],[1,0,0,1,0,1],True],
                     [[int(1)]*100,[int(1)]*100,True],
                     [[int(-1)]*100,[int(-1)]*100,True],
                     [[a,b,c,d]*10,[a,b,c,d]*10,True],
                     [[-a,-b,-c,-d]*10,[-a,-b,-c,-d]*10,True],
                     [[-a,b,-c,d]*10,[-a,b,-c,d]*10,True],
                     [[a,-b,c,-d]*10,[a,-b,c,-d]*10,True]
                    ]
        
        i = 1
        for case in TestCases:
            [test_data, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data) == Vector(expected_result), expected_equal)
            i = i+1
        print("-Done")

        self.assertRaisesRegex(ValueError, Vector._EMPTY_COORDS, lambda: Vector(coordinates=[]))
        self.assertRaisesRegex(ValueError, Vector._MUST_BE_NUMERIC, lambda: Vector(coordinates=[("a",True)]))
        self.assertRaisesRegex(ValueError, Vector._MUST_BE_NUMERIC, lambda: Vector(coordinates=[None]))
        self.assertRaisesRegex(TypeError,".*object is not iterable", lambda: Vector(coordinates=1))
        
    def test_is_zero(self):
        print("\nTest Vector Is Zero Vector")
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[0],True,True],
                     [[0]*2,True,True],
                     [[0]*5,True,True],
                     [[0]*100,True,True],
                     [[1],False,True],
                     [[1]*2,False,True],
                     [[1]*100,False,True],
                     [[0,0.1]*2,False,True],
                     [[-a,-a]*2,False,True],
                     [[a,-a],False,True],
                    ]
        i = 1
        for case in TestCases:
            [test_data, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data).is_zero() == expected_result, expected_equal)
            i = i+1
        print("-Done")
        
    def test_eq(self):
        print("\nTest Vector Equals")
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[int(1)]*100,[int(1)]*100,True],
                     [[int(-1)]*100,[int(-1)]*100,True],
                     [[a,b,c,d]*10,[a,b,c,d]*10,True],
                     [[-a,-b,-c,-d]*10,[-a,-b,-c,-d]*10,True],
                     [[-a,b,-c,d]*10,[-a,b,-c,d]*10,True],
                     [[a,-b,c,-d]*10,[a,-b,c,-d]*10,True],
                     [[int(0)]*100,[int(1)]*100,False],
                     [[int(-1)]*100,[int(1)]*100,False],
                     [[a,b,c,d]*10,[d,c,b,a]*10,False],
                     [[-a,-b,-c,-d]*10,[a,b,c,d]*10,False],
                     [[-a,b,-c,d]*10,[a,-b,c,-d]*10,False],
                     [[a,-b,c,-d]*10,[-a,b,-c,d]*10,False]
                    ]
        i = 1
        for case in TestCases:
            [test_data, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data) == Vector(expected_result), expected_equal)
            i = i+1
        print("-Done")

    def test_add(self):
        print("\nTest Vector Add")
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[int(0)]*100,[int(0)]*100,[int(0)]*100,True],
                     [[int(1)]*100,[int(0)]*100,[int(1)]*100,True],
                     [[int(0)]*100,[int(1)]*100,[int(1)]*100,True],
                     [[int(1)]*100,[int(1)]*100,[int(2)]*100,True],
                     [[int(-1)]*100,[int(-1)]*100,[int(-2)]*100, True],
                     [[int(1)]*100,[int(-1)]*100,[int(0)]*100, True],
                     [[a,b,c,d]*10,[a,b,c,d]*10,[a+a,b+b,c+c,d+d]*10,True],
                     [[-a,-b,-c,-d]*10,[-a,-b,-c,-d]*10,[-a-a,-b-b,-c-c,-d-d]*10,True],
                     [[-a,b,-c,d]*10,[-a,b,-c,d]*10,[-a-a,b+b,-c-c,d+d]*10,True],
                     [[a,-b,c,-d]*10,[a,-b,c,-d]*10,[a+a,-b-b,c+c,-d-d]*10,True]
                    ]
        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1) + Vector(test_data2) == Vector(expected_result), expected_equal)
            i = i + 1
        print("-Done")
        
    def test_sub(self):
        print("\nTest Vector Subtract")
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[int(0)]*100,[int(0)]*100,[int(0)]*100,True],
                     [[int(1)]*100,[int(0)]*100,[int(1)]*100,True],
                     [[int(0)]*100,[int(1)]*100,[int(-1)]*100,True],
                     [[int(1)]*100,[int(1)]*100,[int(0)]*100,True],
                     [[int(-1)]*100,[int(-1)]*100,[int(0)]*100, True],
                     [[int(1)]*100,[int(-1)]*100,[int(2)]*100, True],
                     [[a,b,c,d]*10,[a,b,c,d]*10,[a-a,b-b,c-c,d-d]*10,True],
                     [[-a,-b,-c,-d]*10,[-a,-b,-c,-d]*10,[-a+a,-b+b,-c+c,-d+d]*10,True],
                     [[-a,b,-c,d]*10,[-a,b,-c,d]*10,[-a+a,b-b,-c+c,d-d]*10,True],
                     [[a,-b,c,-d]*10,[a,-b,c,-d]*10,[a-a,-b+b,c-c,-d+d]*10,True]
                    ]
        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1) - Vector(test_data2) == Vector(expected_result), expected_equal)
            i = i + 1
        print("-Done")

    def test_scalar_multiply(self):
        print("\nTest Vector Scalar Multiply")

        _pi = testVector._pi()
        [a,b,c,d] = testVector._rnd_data()
        
        TestCases = [
                     [[int(0)],0,[int(0)],True],
                     [[int(0)],1,[int(0)],True],
                     [[float(1)],0,[float(0)],True],
                     [[float(1)],1,[float(1)],True],
                     [[int(0)]*100,0,[int(0)]*100,True],
                     [[int(0)]*100,1,[int(0)]*100,True],
                     [[int(1)]*100,0,[int(0)]*100,True],
                     [[int(1)]*100,1,[int(1)]*100,True],
                     [[float(1)]*100,1,[float(1)]*100,True],
                     [[float(1)]*100,0,[float(0)]*100,True],
                     [[a,b,c,d]*10,True,[a,b,c,d]*10,True],
                     [[a,b,c,d]*10,False,[0,0,0,0]*10,True],
                     [[a,b,c,d]*10,1,[a,b,c,d]*10,True],
                     [[a,b,c,d]*10,0,[0,0,0,0]*10,True],
                     [[a,b,c,d]*10,2,[2*a,2*b,2*c,2*d]*10,True],
                     [[True,False,True,False]*10,False,[0,0,0,0]*10,True],
                     [[True,False,True,False]*10,True,[1,0,1,0]*10,True],
                     [[True,False,True,False]*10,0,[0,0,0,0]*10,True],
                     [[True,False,True,False]*10,1,[1,0,1,0]*10,True],
                     [[True,False,True,False]*10,_pi,[_pi,0,_pi,0]*10,True],
                     [[a,b,c,d]*10,_pi,[_pi*a,_pi*b,_pi*c,_pi*d]*10,True]
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1)*test_data2 == Vector(expected_result), expected_equal)
            i = i + 1
        print("-Done")

    def test_vector_multiply(self):
        print("\nTest Vector Vector Multiply")
        _pi = testVector._pi()         
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[int(0)],[int(0)],[int(0)],True],
                     [[int(0)],[int(1)],[int(0)],True],
                     [[float(1)],[float(0)],[float(0)],True],
                     [[float(1)],[float(1)],[float(1)],True],
                     [[int(0)]*100,[int(0)]*100,[int(0)]*100,True],
                     [[int(0)]*100,[int(1)]*100,[int(0)]*100,True],
                     [[int(1)]*100,[int(0)]*100,[int(0)]*100,True],
                     [[int(1)]*100,[int(1)]*100,[int(1)]*100,True],
                     [[float(1)]*100,[float(1)]*100,[float(1)]*100,True],
                     [[float(1)]*100,[float(0)]*100,[float(0)]*100,True],
                     [[a,b,c,d]*10,[True]*40,[a,b,c,d]*10,True],
                     [[a,b,c,d]*10,[False]*40,[0,0,0,0]*10,True],
                     [[a,b,c,d]*10,[1]*40,[a,b,c,d]*10,True],
                     [[a,b,c,d]*10,[0]*40,[0,0,0,0]*10,True],
                     [[a,b,c,d]*10,[a,b,c,d]*10,[a*a,b*b,c*c,d*d]*10,True],
                     [[True,False,True,False]*10,[False]*40,[0,0,0,0]*10,True],
                     [[True,False,True,False]*10,[True]*40,[1,0,1,0]*10,True],
                     [[True,False,True,False]*10,[0]*40,[0,0,0,0]*10,True],
                     [[True,False,True,False]*10,[1]*40,[1,0,1,0]*10,True],
                     [[True,False,True,False]*10,[_pi]*40,[_pi,0,_pi,0]*10,True],
                     [[a,b,c,d]*10,[_pi]*40,[a*_pi,b*_pi,c*_pi,d*_pi]*10,True]
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1)*Vector(test_data2) == Vector(expected_result), expected_equal)
            i = i + 1
        print("-Done")

        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([1,1]) * Vector([1,2,3]))
        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([1,1,1,1]) * Vector([1,2,3]))
        
    def test_vector_norm(self):
        print("\nTest Vector norm (magnitude)")
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[int(0)],0,True],
                     [[int(0)]*2,0,True],
                     [[int(-0)]*2,0,True],
                     [[float(0)],0,True],
                     [[float(0)]*2,0,True],
                     [[float(-0)]*2,0,True],
                     [[int(1)],(math.sqrt(1)),True],
                     [[int(1)]*2,(math.sqrt(2)),True],
                     [[int(1)]*3,(math.sqrt(3)),True],
                     [[int(1)]*4,(math.sqrt(4)),True],
                     [[int(-1)]*4,(math.sqrt(4)),True],
                     [[float(1)],(math.sqrt(1)),True],
                     [[float(1)]*2,(math.sqrt(2)),True],
                     [[float(1)]*3,(math.sqrt(3)),True],
                     [[float(1)]*4,(math.sqrt(4)),True],
                     [[float(-1)]*4,(math.sqrt(4)),True],
                     [[a,-b,c,-d],(math.sqrt(a*a + b*b + c*c + d*d)),True],
                     [[-a,b,-c,d],(math.sqrt(a*a + b*b + c*c + d*d)),True],
                     [[a,b,c,d],(math.sqrt(a*a + b*b + c*c + d*d)),True],
                     [[-a,-b,-c,-d],(math.sqrt(a*a + b*b + c*c + d*d)),True]
                    ]

        i = 1
        for case in TestCases:
            [test_data, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data).norm() == expected_result, expected_equal)
            i = i +1
        print("-Done")

    def test_vector_direction(self):
        print("\nTest Vector direction (unit vector)")
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[int(1)]*1,[x *(1/(math.sqrt(1))) for x in [int(1)]*1],True],
                     [[int(1)]*2,[x *(1/(math.sqrt(2))) for x in [int(1)]*2],True],
                     [[int(1)]*3,[x *(1/(math.sqrt(3))) for x in [int(1)]*3],True],
                     [[int(1)]*5,[x *(1/(math.sqrt(5))) for x in [int(1)]*5],True],
                     [[int(1)]*100,[x *(1/(math.sqrt(100))) for x in [int(1)]*100],True],
                     [[True]*100,[x *(1/(math.sqrt(100))) for x in [int(1)]*100],True],
                     [[float(1)]*1,[x *(1/(math.sqrt(1))) for x in [float(1)]*1],True],
                     [[float(1)]*3,[x *(1/(math.sqrt(3))) for x in [float(1)]*3],True],
                     [[float(1)]*5,[x *(1/(math.sqrt(5))) for x in [float(1)]*5],True],
                     [[float(1)]*100,[x *(1/(math.sqrt(100))) for x in [float(1)]*100],True],
                     [[float(-1)]*1,[x *(-1/(math.sqrt(1))) for x in [float(1)]*1],True],
                     [[float(-1)]*5,[x *(-1/(math.sqrt(5))) for x in [float(1)]*5],True],
                     [[float(-1)]*100,[x *(-1/(math.sqrt(100))) for x in [float(1)]*100],True],
                     [[a,-b,c,-d],[x *(float(1)/(math.sqrt(a*a + b*b + c*c + d*d))) for x in [a,-b,c,-d]],True],
                     [[-a,b,-c,d],[x *(float(1)/(math.sqrt(a*a + b*b + c*c + d*d))) for x in [-a,b,-c,d]],True],
                     [[a,b,c,d],[x *(float(1)/(math.sqrt(a*a + b*b + c*c + d*d))) for x in [a,b,c,d]],True],
                     [[-a,-b,-c,-d],[x *(float(1)/(math.sqrt(a*a + b*b + c*c + d*d))) for x in [-a,-b,-c,-d]],True]
                    ]

        i = 1
        for case in TestCases:
            [test_data, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data).direction() == Vector(expected_result), expected_equal)
            i = i +1
        print("-Done")

        self.assertRaisesRegex(ZeroDivisionError, Vector._NO_DIR_FOR_ZERO_VEC, lambda:Vector([int(0)]).direction())
        self.assertRaisesRegex(ZeroDivisionError, Vector._NO_DIR_FOR_ZERO_VEC, lambda:Vector([int(0)]*5).direction())
        self.assertRaisesRegex(ZeroDivisionError, Vector._NO_DIR_FOR_ZERO_VEC, lambda:Vector([float(0)]).direction())
        self.assertRaisesRegex(ZeroDivisionError, Vector._NO_DIR_FOR_ZERO_VEC, lambda:Vector([float(0)]*50).direction())
        self.assertRaisesRegex(ZeroDivisionError, Vector._NO_DIR_FOR_ZERO_VEC, lambda:Vector([False]).direction())

    def test_vector_dot(self):
        print("\nTest Vector dot product")
        _pi = testVector._pi()        
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[int(0)],[int(0)],int(0),True],
                     [[int(0)],[int(1)],int(0),True],
                     [[float(1)],[float(0)],float(0),True],
                     [[float(1)],[float(1)],float(1),True],
                     [[int(0)]*100,[int(0)]*100,int(0),True],
                     [[int(0)]*100,[int(1)]*100,int(0),True],
                     [[int(1)]*100,[int(0)]*100,int(0),True],
                     [[int(1)]*100,[int(1)]*100,int(100),True],
                     [[float(1)]*100,[float(1)]*100,float(100),True],
                     [[float(1)]*100,[float(0)]*100,float(0),True],
                     [[a,b,c,d]*10,[True]*40,sum([a,b,c,d]*10),True],
                     [[a,b,c,d]*10,[False]*40,float(0),True],
                     [[a,b,c,d]*10,[1]*40,sum([a,b,c,d]*10),True],
                     [[a,b,c,d]*10,[0]*40,float(0),True],
                     [[a,b,c,d]*10,[a,b,c,d]*10,sum([a*a,b*b,c*c,d*d]*10),True],
                     [[True,False,True,False]*10,[False]*40,0,True],
                     [[True,False,True,False]*10,[True]*40,20,True],
                     [[True,False,True,False]*10,[0]*40,0,True],
                     [[True,False,True,False]*10,[1]*40,20,True],
                     [[True,False,True,False]*10,[_pi]*40,sum([_pi,0,_pi,0]*10),True],
                     [[a,b,c,d]*10,[_pi]*40,sum([a*_pi,b*_pi,c*_pi,d*_pi]*10),True]
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1).dot(Vector(test_data2)) == expected_result, expected_equal)
            i = i + 1
        print("-Done")

        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([0]).dot(Vector([1,2,3])))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).dot(True))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).dot(int(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).dot(float(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).dot('A'))

    def test_vector_angle(self):
        print("\nTest Vector angle between vectors")
        [a,b,c,d] = testVector._rnd_data()
        half_pi = round(testVector._pi(float(0.5)),7) 
        _pi = round(testVector._pi(),7)
        TestCases = [
                     [[1,1],[1,1],0,True],
                     [[1]*3,[1]*3,0,True],
                     [[0,0,1],[0,0,2],0,True],
                     [[0,a],[a,0],half_pi,True], # at right angles
                     [[b,0],[0,-b],half_pi,True],
                     [[0,-c],[-c,0],half_pi,True],
                     [[-d,0],[0,d],half_pi,True],
                     [[1,0,1],[0,1,0],half_pi,True], 
                     [[1,0,0],[0,1,0],half_pi,True], 
                     [[a,0,a],[0,a,0],half_pi,True], 
                     [[b,0,0],[0,b,0],half_pi,True], 
                     [[1,1,1],[-1,-1,-1],_pi,True], 
                     [[c,c,c],[-c,-c,-c],_pi,True]
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(round(Vector(test_data1).angle(Vector(test_data2)),7) == expected_result, expected_equal)
            i = i + 1
        print("-Done")

        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([1,1]).angle(Vector([1,2,3])))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(True))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(int(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(float(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle('A'))
        self.assertRaisesRegex(ValueError, Vector._NO_ANGLE_FOR_ZERO_VEC, lambda: Vector([0,0]).angle(Vector([0,0])))
        self.assertRaisesRegex(ValueError, Vector._NO_ANGLE_FOR_ZERO_VEC, lambda: Vector([0]).angle(Vector([0])))
        self.assertRaisesRegex(ValueError, Vector._NO_ANGLE_FOR_ZERO_VEC, lambda: Vector([0,0]).angle(Vector([1,1])))
        self.assertRaisesRegex(ValueError, Vector._NO_ANGLE_FOR_ZERO_VEC, lambda: Vector([1,1]).angle(Vector([0,0])))

    def test_vector_parallel(self):
        print("\nTest Vector parallel to other vector")
        [a,b,c,d] = testVector._rnd_data()
        TestCases = [
                     [[1,1],[1,1],True,True],
                     [[1]*3,[1]*3,True,True],
                     [[-1,-1],[-1,-1],True,True],
                     [[-1]*3,[-1]*3,True,True],
                     [[0,0,1],[0,0,2],True,True],
                     [[0,a],[a,0],False,True], # at right angles
                     [[b,0],[0,-b],False,True],
                     [[0,-c],[-c,0],False,True],
                     [[-d,0],[0,d],False,True],
                     [[1,0,1],[0,1,0],False,True], 
                     [[1,0,0],[0,1,0],False,True], 
                     [[a,0,a],[0,a,0],False,True], 
                     [[b,0,0],[0,b,0],False,True], 
                     [[1,1,1],[-1,-1,-1],True,True], 
                     [[c,c,c],[-c,-c,-c],True,True],
                     [[0,0],[0,0],True,True],
                     [[1,1],[0,0],True,True],
                     [[0,0],[1,1],True,True],
                     [[-7.579,-7.88],[22.737,23.64],True,True]
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1).is_parallel_to(Vector(test_data2),1e-7) == expected_result, expected_equal)
            i = i + 1
        print("-Done")

        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([1,1]).angle(Vector([1,2,3])))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(True))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(int(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(float(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle('A'))
        
    def test_vector_orthogonal(self):
        print("\nTest Vector or orthogonal to another vector")
        [a,b,c,d] = testVector._rnd_data()
        half_pi = round(testVector._pi(float(0.5)),7) 
        _pi = round(testVector._pi(),7)
        TestCases = [
                     [[1,1],[1,1],False,True],
                     [[1]*3,[1]*3,False,True],
                     [[0,0,1],[0,0,2],False,True],
                     [[0,a],[a,0],True,True], # at right angles
                     [[b,0],[0,-b],True,True],
                     [[0,-c],[-c,0],True,True],
                     [[-d,0],[0,d],True,True],
                     [[1,0,1],[0,1,0],True,True], 
                     [[1,0,0],[0,1,0],True,True], 
                     [[a,0,a],[0,a,0],True,True], 
                     [[b,0,0],[0,b,0],True,True], 
                     [[1,1,1],[-1,-1,-1],False,True], 
                     [[c,c,c],[-c,-c,-c],False,True],
                     [[0,0],[0,0],True,True],
                     [[1,1],[0,0],True,True],
                     [[0,0],[1,1],True,True]
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1).is_orthogonal_to(Vector(test_data2)) == expected_result, expected_equal)
            i = i + 1
        print("-Done")

        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([1,1]).angle(Vector([1,2,3])))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(True))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(int(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle(float(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).angle('A'))

    def test_vector_projection(self):
        print("\nTest Vector projection onto another vector")
        [a,b,c,d] = testVector._rnd_data()
        # based on a standard 3-4-5
        TestCases = [
                     [[4,3],[1,0],[4,0],True],
                     [[-4,-3],[1,0],[-4,0],True],
                     [[4*a,3*a],[a*b,0],[4*a,0],True],
                     [[-4*a,-3*a],[a*b,0],[-4*a,0],True]
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1).projection_onto(Vector(test_data2)) == Vector(expected_result), expected_equal)
            i = i + 1
        print("-Done")

        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([1,1]).projection_onto(Vector([1,2,3])))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).projection_onto(True))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).projection_onto(int(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).projection_onto(float(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).projection_onto('A'))
        self.assertRaisesRegex(ValueError, Vector._NO_PROJECTION_FOR_ZERO_VEC, lambda: Vector([0,0]).projection_onto(Vector([0,0])))
        self.assertRaisesRegex(ValueError, Vector._NO_PROJECTION_FOR_ZERO_VEC, lambda: Vector([0]).projection_onto(Vector([0])))
        self.assertRaisesRegex(ValueError, Vector._NO_PROJECTION_FOR_ZERO_VEC, lambda: Vector([0,0]).projection_onto(Vector([1,1])))
        self.assertRaisesRegex(ValueError, Vector._NO_PROJECTION_FOR_ZERO_VEC, lambda: Vector([1,1]).projection_onto(Vector([0,0])))

    def test_vector_orth_projection(self):
        print("\nTest Vector orthogonal projection onto another vector")
        [a,b,c,d] = testVector._rnd_data()
        # based on a standard 3-4-5
        TestCases = [
                     [[4,3],[1,0],[0,3],True],
                     [[-4,-3],[1,0],[0,-3],True],
                     [[4*a,3*a],[a*b,0],[0,3*a],True],
                     [[-4*a,-3*a],[a*b,0],[0,-3*a],True],
                     [[4,3],[-1,0],[0,3],True],
                     [[-4,-3],[-1,0],[0,-3],True],
                     [[4*a,3*a],[-a*b,0],[0,3*a],True],
                     [[-4*a,-3*a],[-a*b,0],[0,-3*a],True]
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1).orth_projection_onto(Vector(test_data2)) == Vector(expected_result), expected_equal)
            i = i + 1
        print("-Done")

        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([1,1]).orth_projection_onto(Vector([1,2,3])))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).orth_projection_onto(True))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).orth_projection_onto(int(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).orth_projection_onto(float(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).orth_projection_onto('A'))
        self.assertRaisesRegex(ValueError, Vector._NO_PROJECTION_FOR_ZERO_VEC, lambda: Vector([0,0]).orth_projection_onto(Vector([0,0])))
        self.assertRaisesRegex(ValueError, Vector._NO_PROJECTION_FOR_ZERO_VEC, lambda: Vector([0]).orth_projection_onto(Vector([0])))
        self.assertRaisesRegex(ValueError, Vector._NO_PROJECTION_FOR_ZERO_VEC, lambda: Vector([0,0]).orth_projection_onto(Vector([1,1])))
        self.assertRaisesRegex(ValueError, Vector._NO_PROJECTION_FOR_ZERO_VEC, lambda: Vector([1,1]).orth_projection_onto(Vector([0,0])))

    def test_vector_cross_product(self):
        print("\nTest Vector cross product")
        TestCases = [
                     [[1,0,0],[0,1,0],[0,0,1],True],
                     [[0,1,0],[1,0,0],[0,0,-1],True],
                     [[-1,0,0],[0,1,0],[0,0,-1],True],
                     [[0,1,0],[-1,0,0],[0,0,1],True],            
                     [[-1,0,0],[0,0,1],[0,1,0],True],
                     [[0,0,1],[-1,0,0],[0,-1,0],True],
                     [[0,0,0],[0,0,0],[0,0,0],True],
                     [[1,1,1],[1,-1,-1],[0,2,-2],True],
                     [[1,-1,-1],[1,1,1],[0,-2,2],True],
                    ]        
        i = 1
        for case in TestCases:
            [test_data1, test_data2, expected_result, expected_equal] = case
            print(i,", ",end='')
            self.assertEqual(Vector(test_data1).cross_product(Vector(test_data2)) == Vector(expected_result), expected_equal)
            i = i + 1
            
        # verify as orthogonal
        [a,b,c,d] = testVector._rnd_data()
        _pi = testVector._pi()         
        TestCases = [
                     [[a,0,0],[0,a,0],True],
                     [[0,b,0],[b,0,0],True],
                     [[-c,0,0],[0,c,0],True],
                     [[0,d,0],[-d,0,0],True],            
                     [[-a,0,0],[0,0,a],True],
                     [[0,0,b],[-b,0,0],True],
                    ]        

        for case in TestCases:
            [test_data1, test_data2, expected_equal] = case
            print(i,", ",end='')
            
            v1 = Vector(test_data1).cross_product(Vector(test_data2))
            a1 = v1.angle(Vector(test_data1)) + v1.angle(Vector(test_data2))
            v2 = Vector(test_data2).cross_product(Vector(test_data1))
            a2 = v1.angle(Vector(test_data1)) + v1.angle(Vector(test_data2))
            
            self.assertEqual((a1 - _pi) < 1e-10, expected_equal)
            self.assertEqual((a2 - _pi) < 1e-10, expected_equal)
            i = i + 1
        print("-Done")
        self.assertRaisesRegex(ValueError, Vector._EQUAL_DIM, lambda: Vector([1,2]).cross_product(Vector([1,2,3])))
        self.assertRaisesRegex(ValueError, Vector._DIM_ERROR, lambda: Vector([1]).cross_product(Vector([1])))
        self.assertRaisesRegex(ValueError, Vector._DIM_ERROR, lambda: Vector([1,2,3,4]).cross_product(Vector([1,2,3,4])))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).cross_product(True))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).cross_product(int(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).cross_product(float(0)))
        self.assertRaisesRegex(TypeError, Vector._MUST_BE_VECTOR, lambda: Vector([0]).cross_product('A'))


In [47]:
#
# Run A Single Test
#
test_to_run = "test_vector_cross_product"
suite = unittest.TestSuite()
suite.addTest(testVector(test_to_run))
runner = unittest.TextTestRunner()
runner.run(suite)

.


Test Vector cross product
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , -Done



----------------------------------------------------------------------
Ran 1 test in 0.022s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [48]:
#
# Run All Tests.
#
tests = testVector()
suite = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(suite)

................


Test Vector Add
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , -Done

Test Vector Equals
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , -Done

Test Vector Init
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , -Done

Test Vector Is Zero Vector
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , -Done

Test Vector Scalar Multiply
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , -Done

Test Vector Subtract
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , -Done

Test Vector angle between vectors
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , -Done

Test Vector cross product
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , -Done

Test Vector direction (unit vector)
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , -Done

Test Vector dot product
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , -Done

Test Vector Vector Multiply
1 , 2 , 3 , 4 , 5 , 6 ,


----------------------------------------------------------------------
Ran 16 tests in 0.191s

OK


<unittest.runner.TextTestResult run=16 errors=0 failures=0>